In [1]:
import torch

if torch.cuda.is_available():    

    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80
Sun May  2 08:56:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    27W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                           

In [2]:
!pip install transformers==4.2.1
!pip install farasapy
!pip install pyarabic
!git clone https://github.com/aub-mind/arabert

     |████████████████████████████████| 1.8MB 5.8MB/s 
     |████████████████████████████████| 2.9MB 31.1MB/s 
     |████████████████████████████████| 901kB 32.5MB/s 
     |████████████████████████████████| 112kB 5.5MB/s 
  Created wheel for pyarabic: filename=PyArabic-0.6.10-cp37-none-any.whl size=113324 sha256=6f470330a8d6e1c88f0b3443b99e5bc9a33e31441b585acb0bc5b1ee37035600
  Stored in directory: /root/.cache/pip/wheels/10/b8/f5/b7c1a50e6efb83544844f165a9b134afe7292585465e29b61d
Successfully built pyarabic
Cloning into 'arabert'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (313/313), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 527 (delta 165), reused 225 (delta 82), pack-reused 214
Receiving objects: 100% (527/527), 4.86 MiB | 14.67 MiB/s, done.
Resolving deltas: 100% (288/288), done.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import string
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from sklearn.model_selection import train_test_split

In [5]:
DATA_COLUMN = "text"
LABEL_COLUMN = "label"

In [6]:
folder = 'drive/MyDrive/BuildUp/UNDPLeb/tweets_data/data4/'
name = 'SemEval2017-task4-train.subtask-A.arabic.txt'
filename = folder + name

df1 = pd.read_csv(filename, sep='\t', header=None, names = ['ID', LABEL_COLUMN, DATA_COLUMN], encoding='utf-8')
df1 = df1.loc[:, [DATA_COLUMN, LABEL_COLUMN]]
df1.loc[:, LABEL_COLUMN] = df1.loc[:, LABEL_COLUMN].str.replace('positive', 'POS')
df1.loc[:, LABEL_COLUMN] = df1.loc[:, LABEL_COLUMN].str.replace('negative', 'NEG')
df1.loc[:, LABEL_COLUMN] = df1.loc[:, LABEL_COLUMN].str.replace('neutral', 'NEU')

name = 'SemEval2017-task4-test.subtask-A.arabic.txt'
filename = folder + name

df2 = pd.read_csv(filename, sep='\t', header=None, names = ['ID', LABEL_COLUMN, DATA_COLUMN], encoding='utf-8')
df2 = df2.loc[:, [DATA_COLUMN, LABEL_COLUMN]]
df2.loc[:, LABEL_COLUMN] = df2.loc[:, LABEL_COLUMN].str.replace('positive', 'POS')
df2.loc[:, LABEL_COLUMN] = df2.loc[:, LABEL_COLUMN].str.replace('negative', 'NEG')
df2.loc[:, LABEL_COLUMN] = df2.loc[:, LABEL_COLUMN].str.replace('neutral', 'NEU')

In [7]:
df = pd.concat([df1, df2], ignore_index=True)

In [8]:
exclude = set(string.punctuation)

for i in range(len(df)):
    text = df.loc[i, DATA_COLUMN]
    text = ''.join(ch for ch in text if ch not in exclude)
    text = ''.join(filter(lambda x: x==' ' or x not in string.printable , text))
    df.loc[i, DATA_COLUMN] = text.strip()

In [9]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
label_list = ['NEG', 'POS', 'NEU']

In [10]:
from torch.utils.data import Dataset
from arabert.preprocess import ArabertPreprocessor
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, precision_score , recall_score

In [11]:
model_name = 'aubmindlab/bert-base-arabertv02'
task_name = 'classification'
max_len = 280

In [12]:
arabert_prep = ArabertPreprocessor(model_name.split("/")[-1])

train_df.loc[:, DATA_COLUMN] = train_df.loc[:, DATA_COLUMN].apply(lambda x:   arabert_prep.preprocess(x))
test_df.loc[:, DATA_COLUMN] = test_df.loc[:, DATA_COLUMN].apply(lambda x:   arabert_prep.preprocess(x))  

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [13]:
test_df[DATA_COLUMN]

3470                  احمد الشيخ مظبطنى فى الفنتازى والله
1608    توقعت عتاب او بيونسيه او جنيفر لوبيز حتى كيني ...
8222    من فضلك ريتويت سوا رامي عياش و مايا دياب بدون ...
7410           شيخ ممنوع في باكستان يعظ في مساجد بريطانيا
7306    البقرة د وأجندة اليمين الإلحادي المتصهين الإره...
                              ...                        
868     بتصفيات مونديالم كانت الطائفهي ارض العراق التي...
2304    يعني بقول اهلي و ريال مدريد و مانشستر يونايتد ...
323     المرصدالايرانيالأخبار الكونغرس يشرع برفض عضوية...
8507      الصباحالعربيريكى مارتن يعلن خطوبته من رسام سورى
4502    اردوغانالقدس ما تحدث به اردوغان في اسرائيل بشأ...
Name: text, Length: 1891, dtype: object

In [14]:
class BERTDataset(Dataset):
    def __init__(self, text, target, model_name, max_len, label_map):
      super(BERTDataset).__init__()
      self.text = text
      self.target = target
      self.tokenizer_name = model_name
      self.tokenizer = AutoTokenizer.from_pretrained(model_name)
      self.max_len = max_len
      self.label_map = label_map
      

    def __len__(self):
      return len(self.text)

    def __getitem__(self,item):
      text = str(self.text[item])
      text = " ".join(text.split())


        
      input_ids = self.tokenizer.encode(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          truncation='longest_first'
      )     
    
      attention_mask = [1] * len(input_ids)

      # Zero-pad up to the sequence length.
      padding_length = self.max_len - len(input_ids)
      input_ids = input_ids + ([self.tokenizer.pad_token_id] * padding_length)
      attention_mask = attention_mask + ([0] * padding_length)    
      
      return InputFeatures(input_ids=input_ids, attention_mask=attention_mask, label=self.label_map[self.target[item]])

In [15]:
label_map = { v:index for index, v in enumerate(label_list) }
print(label_map)

train_dataset = BERTDataset(train_df[DATA_COLUMN].to_list(),train_df[LABEL_COLUMN].to_list(),model_name,max_len,label_map)
test_dataset = BERTDataset(test_df[DATA_COLUMN].to_list(),test_df[LABEL_COLUMN].to_list(),model_name,max_len,label_map)

{'NEG': 0, 'POS': 1, 'NEU': 2}


In [16]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(label_map))

In [17]:
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  print(classification_report(p.label_ids,preds))
  print(confusion_matrix(p.label_ids,preds))

  macro_f1_pos_neg = f1_score(p.label_ids,preds,average='macro',labels=[0,1])
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {
      'macro_f1' : macro_f1,
      'macro_f1_pos_neg' : macro_f1_pos_neg,  
      'macro_precision': macro_precision,
      'macro_recall': macro_recall,
      'accuracy': acc
  }

In [18]:
training_args = TrainingArguments("./train")
training_args.evaluate_during_training = True
training_args.adam_epsilon = 1e-8
training_args.learning_rate = 1e-5
training_args.fp16 = True
training_args.per_device_train_batch_size = 16
training_args.per_device_eval_batch_size = 16
training_args.gradient_accumulation_steps = 2
training_args.num_train_epochs= 8


steps_per_epoch = (len(train_df)// (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps))
total_steps = steps_per_epoch * training_args.num_train_epochs
print(steps_per_epoch)
print(total_steps)
#Warmup_ratio
warmup_ratio = 0.1
training_args.warmup_steps = total_steps*warmup_ratio # or you can set the warmup steps directly 

training_args.evaluation_strategy = EvaluationStrategy.EPOCH
# training_args.logging_steps = 200
training_args.save_steps = 100000 #don't want to save any model, there is probably a better way to do this :)
training_args.seed = 42
training_args.disable_tqdm = False
training_args.lr_scheduler_type = 'cosine'

236
1888


In [ ]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_dataset, 
    eval_dataset=test_dataset, 
    model_init=model_init,
    compute_metrics=compute_metrics,
)
trainer.train()

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

Epoch,Training Loss,Validation Loss,Macro F1,Macro F1 Pos Neg,Macro Precision,Macro Recall,Accuracy,Runtime,Samples Per Second
0,No log,0.758904,0.613054,0.604878,0.675763,0.617391,0.652565,109.748700,17.230000
1,No log,0.657611,0.717880,0.747219,0.713383,0.728522,0.717081,109.817500,17.219000
2,0.807000,0.639801,0.729640,0.753581,0.730870,0.732494,0.733474,109.767700,17.227000
3,0.807000,0.670267,0.722924,0.754316,0.719921,0.729972,0.723956,109.782300,17.225000
4,0.459400,0.705901,0.730356,0.748478,0.736187,0.725972,0.731888,109.757800,17.229000
5,0.459400,0.731455,0.733567,0.750732,0.740224,0.728878,0.735590,109.495700,17.270000


              precision    recall  f1-score   support

           0       0.70      0.87      0.78       660
           1       0.74      0.31      0.43       466
           2       0.59      0.68      0.63       765

    accuracy                           0.65      1891
   macro avg       0.68      0.62      0.61      1891
weighted avg       0.66      0.65      0.63      1891

[[573   4  83]
 [ 43 143 280]
 [200  47 518]]
              precision    recall  f1-score   support

           0       0.78      0.79      0.79       660
           1       0.65      0.78      0.71       466
           2       0.71      0.62      0.66       765

    accuracy                           0.72      1891
   macro avg       0.71      0.73      0.72      1891
weighted avg       0.72      0.72      0.72      1891

[[522  24 114]
 [ 24 363  79]
 [122 172 471]]
              precision    recall  f1-score   support

           0       0.76      0.87      0.81       660
           1       0.72      0.68    